<a href="https://colab.research.google.com/github/jrtrj/ImageSharpening_KD/blob/edsr/edsr_Teacher_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/sanghyun-son/EDSR-PyTorch.git
%cd EDSR-PyTorch


Cloning into 'EDSR-PyTorch'...
remote: Enumerating objects: 806, done.
remote: Total 806 (delta 0), reused 0 (delta 0), pack-reused 806 (from 1)
Receiving objects: 100% (806/806), 63.09 MiB | 50.75 MiB/s, done.
Resolving deltas: 100% (516/516), done.
/content/EDSR-PyTorch


for the below pretrained weights,
in the model args add this
args.scale = [2]
args.n_resblocks = 16
args.n_feats = 64


In [ ]:

!wget https://cv.snu.ac.kr/research/EDSR/models/edsr_baseline_x2-1bc95232.pt

--2025-07-02 18:35:04--  https://cv.snu.ac.kr/research/EDSR/models/edsr_baseline_x2-1bc95232.pt
Resolving cv.snu.ac.kr (cv.snu.ac.kr)... 147.46.67.83
Connecting to cv.snu.ac.kr (cv.snu.ac.kr)|147.46.67.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5490680 (5.2M)
Saving to: ‘edsr_baseline_x2-1bc95232.pt’

edsr_baseline_x2-1b 100%[===================>]   5.24M  2.29MB/s    in 2.3s    

2025-07-02 18:35:07 (2.29 MB/s) - ‘edsr_baseline_x2-1bc95232.pt’ saved [5490680/5490680]



for the below pretrained weights,
in the model args add this
args.scale = [4]
args.n_resblocks = 16
args.n_feats = 64

with this i am getting ssim of 90

In [ ]:
!wget https://cv.snu.ac.kr/research/EDSR/models/edsr_baseline_x4-6b446fab.pt

--2025-07-02 19:29:58--  https://cv.snu.ac.kr/research/EDSR/models/edsr_baseline_x4-6b446fab.pt
Resolving cv.snu.ac.kr (cv.snu.ac.kr)... 147.46.67.83
Connecting to cv.snu.ac.kr (cv.snu.ac.kr)|147.46.67.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6081831 (5.8M)
Saving to: ‘edsr_baseline_x4-6b446fab.pt’

edsr_baseline_x4-6b 100%[===================>]   5.80M  2.82MB/s    in 2.1s    

2025-07-02 19:30:01 (2.82 MB/s) - ‘edsr_baseline_x4-6b446fab.pt’ saved [6081831/6081831]



for the below pretrained weights,
in the model args add this
args.scale = [4]
args.n_resblocks = 32
args.n_feats = 256

but using this i am getting only 0.0029
but this more stronger weight

In [ ]:
!wget https://cv.snu.ac.kr/research/EDSR/models/edsr_x4-4f62e9ef.pt

--2025-07-02 19:32:53--  https://cv.snu.ac.kr/research/EDSR/models/edsr_x4-4f62e9ef.pt
Resolving cv.snu.ac.kr (cv.snu.ac.kr)... 147.46.67.83
Connecting to cv.snu.ac.kr (cv.snu.ac.kr)|147.46.67.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172381555 (164M)
Saving to: ‘edsr_x4-4f62e9ef.pt’

edsr_x4-4f62e9ef.pt 100%[===================>] 164.40M  4.86MB/s    in 36s     

2025-07-02 19:33:30 (4.58 MB/s) - ‘edsr_x4-4f62e9ef.pt’ saved [172381555/172381555]



In [ ]:
from PIL import Image
import torch
import torchvision.transforms as T
from skimage.metrics import structural_similarity as compare_ssim
import numpy as np
import sys
import types

sys.path.append('src')
from model import edsr

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load HR image
hr_path = '/content/0004.png'
hr_img = Image.open(hr_path).convert('RGB')

# Make LR image by downscaling by ×4
scale_factor = 4
lr_w, lr_h = hr_img.width // scale_factor, hr_img.height // scale_factor
lr_img = hr_img.resize((lr_w, lr_h), Image.BICUBIC)

# Transform to tensor
to_tensor = T.ToTensor()
lr_tensor = to_tensor(lr_img).unsqueeze(0).to(device) * 255  # model expects [0,255]
hr_tensor = to_tensor(hr_img).unsqueeze(0).to(device) * 255

# Model args (match pretrained model)
args = types.SimpleNamespace()
args.scale = [4]
args.n_resblocks = 16
args.n_feats = 64
args.rgb_range = 255
args.n_colors = 3
args.res_scale = 1

# Load model and weights
model = edsr.EDSR(args).to(device)
ckpt = torch.load('/content/EDSR-PyTorch/edsr_baseline_x4-6b446fab.pt', map_location=device)
model.load_state_dict(ckpt, strict=True)
model.eval()

print(f"LR tensor shape: {lr_tensor.shape}")
print(f"HR tensor shape: {hr_tensor.shape}")
lr_upscaled = lr_img.resize((hr_img.width, hr_img.height), Image.BICUBIC)
lr_upscaled_np = np.array(lr_upscaled).astype(np.float32) / 255.0
hr_np = np.array(hr_img).astype(np.float32) / 255.0

ssim_bicubic = compare_ssim(hr_np, lr_upscaled_np, channel_axis=-1, data_range=1.0)
print(f"SSIM bicubic: {ssim_bicubic:.4f}")

with torch.no_grad():
    output = model(lr_tensor)
    print(f"Output tensor shape: {output.shape}")
    output = output.clamp(0, args.rgb_range) / args.rgb_range  # back to [0,1]
    hr_tensor_norm = hr_tensor / args.rgb_range

    # Convert to numpy
    sr_img_np = output.squeeze(0).permute(1,2,0).cpu().numpy()
    hr_img_np = hr_tensor_norm.squeeze(0).permute(1,2,0).cpu().numpy()

    # Ensure same size
    min_h = min(sr_img_np.shape[0], hr_img_np.shape[0])
    min_w = min(sr_img_np.shape[1], hr_img_np.shape[1])
    sr_img_np = sr_img_np[:min_h, :min_w, :]
    hr_img_np = hr_img_np[:min_h, :min_w, :]

    # Compute SSIM safely
    min_side = min(hr_img_np.shape[0], hr_img_np.shape[1])
    win_size = min(7, min_side)
    if win_size %2==0: win_size-=1
    if win_size<3: win_size=3

    ssim_score = compare_ssim(
        hr_img_np, sr_img_np,
        data_range=1.0,
        channel_axis=-1,
        win_size=win_size
    )
    print(f'SSIM (win_size={win_size}): {ssim_score:.4f}')



LR tensor shape: torch.Size([1, 3, 336, 510])
HR tensor shape: torch.Size([1, 3, 1344, 2040])
SSIM bicubic: 0.8753
Output tensor shape: torch.Size([1, 3, 1344, 2040])
SSIM (win_size=7): 0.9014
